In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# Obtain github paths for data downloads
examples_url = "https://github.com/amazon-science/esci-data/raw/main/shopping_queries_dataset/shopping_queries_dataset_examples.parquet"
products_url = "https://github.com/amazon-science/esci-data/raw/main/shopping_queries_dataset/shopping_queries_dataset_products.parquet"

# Download examples with query-product mappings
examples_response = requests.get(examples_url)
with open("../data/raw/examples.parquet", "wb") as f:
    f.write(examples_response.content)
del examples_response # Free up memory before next download

# Download products with product details  
products_response = requests.get(products_url)
with open("../data/raw/products.parquet", "wb") as f:
    f.write(products_response.content)
del products_response # Free up memory from large download

In [2]:
# Load datasets
df_examples = pd.read_parquet("../data/raw/examples.parquet")
df_products = pd.read_parquet("../data/raw/products.parquet")

# Merge on product_locale and product_id, per github repo instructions
df_examples_products = pd.merge(
    df_examples, 
    df_products,
    how='left',
    left_on=['product_locale', 'product_id'],
    right_on=['product_locale', 'product_id']
)

# Preliminary look at the merged dataset
print(f"Dataset shape: {df_examples_products.shape}")
df_examples_products.head()

Dataset shape: (2621288, 14)


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White


In [3]:
# Filter for task 1 and target criteria
df_task1 = df_examples_products[df_examples_products["small_version"] == 1]

# Look at the down-sized small version dataset
print(f"Dataset small_version shape: {df_task1.shape}")

# Apply filters for US locale and Exact matches only
df_filtered = df_task1[
    (df_task1['product_locale'] == 'us') & 
    (df_task1['esci_label'] == 'E')
]

# Look at the further filtered dataset
print(f"Dataset filtered shape: {df_filtered.shape}")
print(f"Unique queries: {df_filtered['query'].nunique()}")

# Another quick look at the data
df_filtered.head()

Dataset small_version shape: (1118011, 14)
Dataset filtered shape: (261527, 14)
Unique queries: 29843


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
17,17,!awnmower tires without rims,1,B08L3B9B9P,us,E,1,1,train,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,Please check your existing tire Sidewall for t...,MaxAuto,None
20,20,!awnmower tires without rims,1,B07C1WZG12,us,E,1,1,train,(Set of 2) 15x6.00-6 Husqvarna/Poulan Tire Whe...,No fuss. Just take off your old assembly and r...,Tire size:15x6.00-6 Ply: 4 Tubeless\n6x4.5 Whe...,Antego Tire & Wheel,Husqvarna Silver
21,21,!awnmower tires without rims,1,B077QMNXTS,us,E,1,1,train,MaxAuto 2 Pcs 16x6.50-8 Lawn Mower Tire for Ga...,<br>Tire Specifications:<br> 1. Material: Rubb...,"Set of 2 16X6.50-8, 16x6.50x8, 16-6.50-8 Lawn ...",MaxAuto,Black
23,23,!awnmower tires without rims,1,B06XX6BM2R,us,E,1,1,train,"MARASTAR 21446-2PK 15x6.00-6"" Front Tire Assem...",None,Tire: 2 pack 15x6. 00-6 tube-type turf SAVER t...,MARASTAR,None
26,26,!awnmower tires without rims,1,B0089RNSNM,us,E,1,1,train,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Set of 2 Honda OEM Rear Wheels\nReplaces 42710...,Honda,None


* Notice '!' exclamation point instead of 'l' for the first letter in lawn mower for query col.
  * Some text cleanup likely

In [4]:
# Further look at the data before down-sampling
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 261527 entries, 17 to 2618569
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   example_id            261527 non-null  int64 
 1   query                 261527 non-null  object
 2   query_id              261527 non-null  int64 
 3   product_id            261527 non-null  object
 4   product_locale        261527 non-null  object
 5   esci_label            261527 non-null  object
 6   small_version         261527 non-null  int64 
 7   large_version         261527 non-null  int64 
 8   split                 261527 non-null  object
 9   product_title         261527 non-null  object
 10  product_description   134067 non-null  object
 11  product_bullet_point  231366 non-null  object
 12  product_brand         249806 non-null  object
 13  product_color         184355 non-null  object
dtypes: int64(4), object(10)
memory usage: 29.9+ MB


* Many missing values for the 'product_' related columns:
  * ~51% for 'product_description' are missing!
  * ~11% for 'product_bullet_point' are missing
  * ~4% for 'product_brand' are missing
  * ~29% for 'product_color' are missing
* I find it unlikely that a company would have such missing information for their product catalog:
  * Assumption to drop null values before creating ~500 row / 50 unique query sample dataset

In [9]:
# Save current filtered dataset
df_filtered.to_parquet("../data/processed/df_filtered.parquet")

## Checkpoint 1

In [10]:
# Checkpoint to reload filtered dataset
df_filtered = pd.read_parquet("../data/processed/df_filtered.parquet")

In [5]:
# Drop null values for the 'product_' related columns
df_dropped = df_filtered.dropna(subset=['product_description', 'product_bullet_point', 'product_brand', 'product_color']).reset_index(drop=True)

# Look at the data again before trying down-sampling
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93296 entries, 0 to 93295
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   example_id            93296 non-null  int64 
 1   query                 93296 non-null  object
 2   query_id              93296 non-null  int64 
 3   product_id            93296 non-null  object
 4   product_locale        93296 non-null  object
 5   esci_label            93296 non-null  object
 6   small_version         93296 non-null  int64 
 7   large_version         93296 non-null  int64 
 8   split                 93296 non-null  object
 9   product_title         93296 non-null  object
 10  product_description   93296 non-null  object
 11  product_bullet_point  93296 non-null  object
 12  product_brand         93296 non-null  object
 13  product_color         93296 non-null  object
dtypes: int64(4), object(10)
memory usage: 10.0+ MB


* Full dataset with no missing values has ~93k records:
  * Try ~500 row / 50 unique query sample dataset creation

In [ ]:
['product_description', 'product_bullet_point', 'product_brand', 'product_color']

In [ ]:
# Create sample dataset: ~500 rows with ~50 unique queries
np.random.seed(2025) # set random seed for reproducibility (like to use the year)
unique_queries = df_dropped['query'].unique()

# Sample 50 queries randomly
n_queries = min(50, len(unique_queries))
sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)

# Get all products for these queries
df_sample = df_dropped[df_dropped['query'].isin(sample_queries)].copy()

# If too many rows, sample proportionally per query
if df_sample.shape[0] > 500:
    target_per_query = max(1, 500 // n_queries)
    df_sample = df_sample.groupby('query').apply(
        lambda x: x.sample(n=min(len(x), target_per_query), random_state=2025)
    ).reset_index(drop=True)
else:
    df_sample = df_sample.reset_index(drop=True)

print(f"Final sample: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

# Look at the sample dataset head
# df_sample[['query', 'product_title']].head()
df_sample.head()

Final sample: 210 rows, 50 queries


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,117965,6 acrylic brushes for nails,4839,B087CJ6ZVV,us,E,1,1,test,"Yolife Acrylic Nail Art Brush, 100% Pure Kolin...",<b>Package Content: </b><p>1* Red Wooden Nail ...,❤️【Ingenious Design】100% Pure Kolinsky Sable N...,Yolife,Wood
1,117966,6 acrylic brushes for nails,4839,B083ZMG2ZX,us,E,1,1,test,Pana USA Acrylic Nail Brush100% Pure Kolinsky ...,PANA Finest Kolinsky Hair Professional Acrylic...,[Design] Size 6 Brush Handle | A smooth and st...,PANA,Mahogany
2,117967,6 acrylic brushes for nails,4839,B07Y55F8HH,us,E,1,1,test,1PCS Eval 100% Kolinsky Sable hair Brushes Acr...,Eval Crimped Shaped Kolinsky Sable Brushes #6 ...,★HIGH QUALITY HAIR:Brushes made of Kolinsky sa...,Eval,Purple
3,137801,a 6 motels,5780,B07MTMP7D5,us,E,1,1,train,"Evocel Galaxy A6 (2018) Case, [Explorer Series...",The Evocel Explorer Series Pro is a phone case...,"[Dependable Protection] Heavy duty, dual layer...",Evocel,Blue
4,137803,a 6 motels,5780,B0796M8RPJ,us,E,1,1,train,Aunifun Replacement Filter & Side Brush Kit fo...,<b>Fit for</b> ILIFE A6 A4 A4s Robot Vacuum Cl...,★Fit for ILIFE A6 A4 A4s Robot Vacuum Cleaner....,Aunifun,Replacement Filter & Side Brush Kit for ILIFE ...


* Only able to sample to 210 rows from non-null ~93k dataset:
  * May need to accept some missing 'product_' values and develop imputation strategy
  * Turn down-sampling into defined function and try different ways to handle missing values

# Put cell below here!

In [23]:
df_dropped['query'].nunique()

20174

In [24]:
df_filtered['query'].nunique()

29843

In [14]:
# Drop null values for everything but 'product_color' related columns
df_color = df_filtered.dropna(subset=['product_description', 'product_bullet_point', 'product_brand']).reset_index(drop=True)

# Look at the data again before trying down-sampling
df_color.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124540 entries, 0 to 124539
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   example_id            124540 non-null  int64 
 1   query                 124540 non-null  object
 2   query_id              124540 non-null  int64 
 3   product_id            124540 non-null  object
 4   product_locale        124540 non-null  object
 5   esci_label            124540 non-null  object
 6   small_version         124540 non-null  int64 
 7   large_version         124540 non-null  int64 
 8   split                 124540 non-null  object
 9   product_title         124540 non-null  object
 10  product_description   124540 non-null  object
 11  product_bullet_point  124540 non-null  object
 12  product_brand         124540 non-null  object
 13  product_color         93296 non-null   object
dtypes: int64(4), object(10)
memory usage: 13.3+ MB


In [15]:
# Create sample dataset: ~500 rows with ~50 unique queries
np.random.seed(2025) # set random seed for reproducibility (like to use the year)
unique_queries = df_color['query'].unique()

# Sample 50 queries randomly
n_queries = min(50, len(unique_queries))
sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)

# Get all products for these queries
df_sample = df_color[df_color['query'].isin(sample_queries)].copy()

# If too many rows, sample proportionally per query
if df_sample.shape[0] > 500:
    target_per_query = max(1, 500 // n_queries)
    df_sample = df_sample.groupby('query').apply(
        lambda x: x.sample(n=min(len(x), target_per_query), random_state=2025)
    ).reset_index(drop=True)
else:
    df_sample = df_sample.reset_index(drop=True)

print(f"Final sample: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

# Look at the sample dataset head
# df_sample[['query', 'product_title']].head()
df_sample.head()

Final sample: 232 rows, 50 queries


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,14967,05 4runner wheel spacers without hub centric,520,B08LV2F2VL,us,E,1,1,train,Rying 4PCS 6x5.5 Wheel Spacers for Tacoma Tund...,<p>Blue Tacoma Tundra 6x5.5 Wheel Spacers</p> ...,Wheel Spacers 6x5.5 to 6x5.5 SPECS: (shown as ...,Rying,None
1,14968,05 4runner wheel spacers without hub centric,520,B083NZF7QW,us,E,1,1,train,Orion Motor Tech 6x5.5 Hubcentric Wheel Spacer...,<p>This set of 4 Orion Motor Tech wheel spacer...,SAFE AND SECURE: This Orion Motor Tech automot...,Orion Motor Tech,None
2,14970,05 4runner wheel spacers without hub centric,520,B07XHFV3V1,us,E,1,1,train,Wheel Spacer Set of 4-6 Lug 1 inch Thick 25mm ...,These set of 4 Wheel Spacers are 1 inch thick ...,"Wheel Spacer Kit of 4 6X5.5"" 1 Inch - Allows e...",AA Ignition,None
3,14971,05 4runner wheel spacers without hub centric,520,B07XCCB368,us,E,1,1,train,Wheel Spacer Set of 4-6 Lug 1.5 inch 38mm Hub ...,These set of 4 Wheel Spacers are 1.5 inch thic...,"Hubcentric Wheel Spacer Kit 6X5.5"" 1.5 Inches ...",AA Ignition,None
4,14972,05 4runner wheel spacers without hub centric,520,B07LC6CYNT,us,E,1,1,train,Wheel Accessories Parts 2 Universal Spacers 12...,Note: Not hub centric. This set is for Univers...,Note: Not hub centric. This set is for Univers...,Wheel Accessories Parts,None


In [ ]:
# Drop null values for everything but 'product_bullet_point' related columns
df_bullet = df_filtered.dropna(subset=['product_description', 'product_brand', 'product_color']).reset_index(drop=True)

# Look at the data again before trying down-sampling
df_bullet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95062 entries, 0 to 95061
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   example_id            95062 non-null  int64 
 1   query                 95062 non-null  object
 2   query_id              95062 non-null  int64 
 3   product_id            95062 non-null  object
 4   product_locale        95062 non-null  object
 5   esci_label            95062 non-null  object
 6   small_version         95062 non-null  int64 
 7   large_version         95062 non-null  int64 
 8   split                 95062 non-null  object
 9   product_title         95062 non-null  object
 10  product_description   95062 non-null  object
 11  product_bullet_point  93296 non-null  object
 12  product_brand         95062 non-null  object
 13  product_color         95062 non-null  object
dtypes: int64(4), object(10)
memory usage: 10.2+ MB


In [17]:
# Create sample dataset: ~500 rows with ~50 unique queries
np.random.seed(2025) # set random seed for reproducibility (like to use the year)
unique_queries = df_bullet['query'].unique()

# Sample 50 queries randomly
n_queries = min(50, len(unique_queries))
sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)

# Get all products for these queries
df_sample = df_bullet[df_bullet['query'].isin(sample_queries)].copy()

# If too many rows, sample proportionally per query
if df_sample.shape[0] > 500:
    target_per_query = max(1, 500 // n_queries)
    df_sample = df_sample.groupby('query').apply(
        lambda x: x.sample(n=min(len(x), target_per_query), random_state=2025)
    ).reset_index(drop=True)
else:
    df_sample = df_sample.reset_index(drop=True)

print(f"Final sample: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

# Look at the sample dataset head
# df_sample[['query', 'product_title']].head()
df_sample.head()

Final sample: 304 rows, 50 queries


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,20856,1 flexible 8x10 mirror not sheet,734,B08C51HX6W,us,E,1,1,train,20 Pieces Mirror Sheets Self Adhesive Non Glas...,<br>Wide range of usage: <br>These self adhesi...,Covered with protective film: these mirrors ar...,BBTO,Clear
1,20868,1 flexible 8x10 mirror not sheet,734,B08F2BKC9F,us,E,1,1,train,Professional Large Double Sided 360° Swivel 1X...,<p><b>This is the Hollywood Standard</b>High-e...,"🔎【Strong 10X Magnification, Magnify Your Beaut...",MIRRORMORE,Vintage Silver
2,20875,1 flexible 8x10 mirror not sheet,734,B01AWXNHT0,us,E,1,1,train,"12"" x 24"" 1/8"" Acrylic Mirror Sheet","12"" x 24"" 1/8"" Acrylic Mirror Sheet",acrylic\nAcrylic mirrored sheet\nCan be cut wi...,SIBE AUTOMATION,Silver Mirror - Without Adhesive Back
3,20879,1 flexible 8x10 mirror not sheet,734,B07173LNJ3,us,E,1,1,train,9 Pieces Flexible Mirror Sheets Self Adhesive ...,<br>Features: <br> <br>Various sizes of self-a...,With protective films: each plastic mirror com...,BBTO,9 Pieces
4,32121,1/4” black buttons for sewing without holes,1140,B07KGGGF7Z,us,E,1,1,train,"Seeking ROAM Buttons 2 Hole, 1/2 Inch, Resin, ...",The perfect button for your perfect project. W...,Our 10 pack of buttons are the perfect size fo...,Seeking ROAM,Black


In [18]:
# Drop null values for everything but 'product_brand' related columns
df_brand = df_filtered.dropna(subset=['product_description', 'product_bullet_point', 'product_color']).reset_index(drop=True)

# Look at the data again before trying down-sampling
df_brand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93550 entries, 0 to 93549
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   example_id            93550 non-null  int64 
 1   query                 93550 non-null  object
 2   query_id              93550 non-null  int64 
 3   product_id            93550 non-null  object
 4   product_locale        93550 non-null  object
 5   esci_label            93550 non-null  object
 6   small_version         93550 non-null  int64 
 7   large_version         93550 non-null  int64 
 8   split                 93550 non-null  object
 9   product_title         93550 non-null  object
 10  product_description   93550 non-null  object
 11  product_bullet_point  93550 non-null  object
 12  product_brand         93296 non-null  object
 13  product_color         93550 non-null  object
dtypes: int64(4), object(10)
memory usage: 10.0+ MB


In [19]:
# Create sample dataset: ~500 rows with ~50 unique queries
np.random.seed(2025) # set random seed for reproducibility (like to use the year)
unique_queries = df_brand['query'].unique()

# Sample 50 queries randomly
n_queries = min(50, len(unique_queries))
sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)

# Get all products for these queries
df_sample = df_brand[df_brand['query'].isin(sample_queries)].copy()

# If too many rows, sample proportionally per query
if df_sample.shape[0] > 500:
    target_per_query = max(1, 500 // n_queries)
    df_sample = df_sample.groupby('query').apply(
        lambda x: x.sample(n=min(len(x), target_per_query), random_state=2025)
    ).reset_index(drop=True)
else:
    df_sample = df_sample.reset_index(drop=True)

print(f"Final sample: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

# Look at the sample dataset head
# df_sample[['query', 'product_title']].head()
df_sample.head()

Final sample: 224 rows, 50 queries


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,15941,08 chevy tahoe wheel not cover,568,B07CKS69CN,us,E,1,1,test,"DYNOFIT 14mmx1.5 Wheel Lug Nuts, 24x Silver 7 ...",<p><b>Size</b></p> <p>Thread pitch: 14mmx1.50/...,Vehicles Select Chart: Dodge Dakota Lincoln Ma...,DYNOFIT,Silver
1,15945,08 chevy tahoe wheel not cover,568,B00O2M424C,us,E,1,1,test,Black Wheel Lug Nut Cap Cover for Chevrolet an...,Compatible/Replacement for Chevy/GM 9593028 95...,For dual thread lug nuts with a 19mm or 3/4 in...,DPAccessories,Black
2,18553,1 32 cars not pullback,655,B08TV1NFZ4,us,E,1,1,train,"Bugatti Divo Diecast Car,Zinc Alloy Casting Mo...",<b> Description</b><br><b>Product Name:</b>1:3...,Material:Zinc Alloy Metal Body Solid Structure...,YHXKJ,Grey
3,18556,1 32 cars not pullback,655,B094N6HP6V,us,E,1,1,train,Car Model X 1:32 Scale Alloy Diecast Pull Back...,<b>Condition:New</b> <br>Battery Included: AG1...,SPECIAL FEATURES: This collectible toy car has...,chengchuang,Blue
4,18562,1 32 cars not pullback,655,B07WZXTBFQ,us,E,1,1,train,TGRCM-CZ 1:36 Scale Benz G63 Car Model for Kid...,<b>TGRCM-CZ Toy Car Description</b><br><br><br...,High Quality and Authorized by G63 : Authorize...,TGRCM-CZ,White


In [20]:
# Drop null values for everything but 'product_description' related columns
df_desc = df_filtered.dropna(subset=['product_bullet_point', 'product_brand', 'product_color']).reset_index(drop=True)

# Look at the data again before trying down-sampling
df_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178908 entries, 0 to 178907
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   example_id            178908 non-null  int64 
 1   query                 178908 non-null  object
 2   query_id              178908 non-null  int64 
 3   product_id            178908 non-null  object
 4   product_locale        178908 non-null  object
 5   esci_label            178908 non-null  object
 6   small_version         178908 non-null  int64 
 7   large_version         178908 non-null  int64 
 8   split                 178908 non-null  object
 9   product_title         178908 non-null  object
 10  product_description   93296 non-null   object
 11  product_bullet_point  178908 non-null  object
 12  product_brand         178908 non-null  object
 13  product_color         178908 non-null  object
dtypes: int64(4), object(10)
memory usage: 19.1+ MB


In [21]:
# Create sample dataset: ~500 rows with ~50 unique queries
np.random.seed(2025) # set random seed for reproducibility (like to use the year)
unique_queries = df_desc['query'].unique()

# Sample 50 queries randomly
n_queries = min(50, len(unique_queries))
sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)

# Get all products for these queries
df_sample = df_desc[df_desc['query'].isin(sample_queries)].copy()

# If too many rows, sample proportionally per query
if df_sample.shape[0] > 500:
    target_per_query = max(1, 500 // n_queries)
    df_sample = df_sample.groupby('query').apply(
        lambda x: x.sample(n=min(len(x), target_per_query), random_state=2025)
    ).reset_index(drop=True)
else:
    df_sample = df_sample.reset_index(drop=True)

print(f"Final sample: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

# Look at the sample dataset head
# df_sample[['query', 'product_title']].head()
df_sample.head()

Final sample: 373 rows, 50 queries


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,11746,0 outdoor stand alone umbrella without base,401,B0811S2LN2,us,E,1,1,train,MEWAY 10ft Outdoor Umbrella Patio Offset Canti...,None,[ SIZE AND USE ]: This outdoor umbrella with 1...,MEWAY,Beige
1,11747,0 outdoor stand alone umbrella without base,401,B081NG9Q95,us,E,1,1,train,Sunnyglade 9Ft Patio Umbrella Outdoor Table Um...,9 feet diameter patio straight umbrella for ex...,"Made of 100% polyester that can be waterproof,...",Sunnyglade,Tan
2,11748,0 outdoor stand alone umbrella without base,401,B083B8QJ44,us,E,1,1,train,BLUU BANYAN 10 FT Patio Offset Umbrella Outdoo...,None,PREMIUM PATIO DÉCOR: The BLUU BANYAN umbrella ...,BLUU,Beige
3,11749,0 outdoor stand alone umbrella without base,401,B083D27277,us,E,1,1,train,wikiwiki H Series Patio Offset Hanging Umbrell...,None,UPGRADED SOLUTION-DYED CANOPY: The WIKIWIKI H ...,wikiwiki,Beige
4,11751,0 outdoor stand alone umbrella without base,401,B0854574VC,us,E,1,1,train,Sunnyglade 10x10Ft Cantilever Patio Umbrella S...,None,【Upgraded Fabric & Innovative Design】This offs...,Sunnyglade,Tan


In [22]:
# Create sample dataset: ~500 rows with ~50 unique queries
np.random.seed(2025) # set random seed for reproducibility (like to use the year)
unique_queries = df_filtered['query'].unique()

# Sample 50 queries randomly
n_queries = min(50, len(unique_queries))
sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)

# Get all products for these queries
df_sample = df_filtered[df_filtered['query'].isin(sample_queries)].copy()

# If too many rows, sample proportionally per query
if df_sample.shape[0] > 500:
    target_per_query = max(1, 500 // n_queries)
    df_sample = df_sample.groupby('query').apply(
        lambda x: x.sample(n=min(len(x), target_per_query), random_state=2025)
    ).reset_index(drop=True)
else:
    df_sample = df_sample.reset_index(drop=True)

print(f"Final sample: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

# Look at the sample dataset head
# df_sample[['query', 'product_title']].head()
df_sample.head()

Final sample: 407 rows, 50 queries


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,1673,#pinkandproud not sorry glitter lipstick,53,B07PGQDJ73,us,E,1,1,train,"Maybelline SuperStay Ink Crayon Lipstick, Matt...",None,Long Lasting Lip Color: Our SuperStay Ink Cray...,Maybelline New York,65 SETTLE FOR MORE
1,1676,#pinkandproud not sorry glitter lipstick,53,B07R7V7XLN,us,E,1,1,train,Coosa Glitter Liquid Lipsticks Set 6 color Dia...,<b>Coosa Glitter Shimmer Diamond shimmer glitt...,High Shimmering Pigments--Diamond shimmer glit...,Coosa,6 PCS
2,1685,#pinkandproud not sorry glitter lipstick,53,B08F7CWDY2,us,E,1,1,train,"Matte Liquid Lipstick Set, Durable Nude Lip Gl...",None,Healthy Beauty : Our lip gloss kit is hypoalle...,LANGMANNI,24PCS
3,1688,#pinkandproud not sorry glitter lipstick,53,B08LYP272Y,us,E,1,1,train,evpct 4Pack Mini Glitter Cigarette Lipstick Se...,<b>Specification: <b><br> ●6 Different Types o...,❤Cool Girl Cigarette Box Design -- This lipsti...,evpct,A-Set04
4,2958,$3what can you buy without siri dollar toy,96,B07GWPVCZK,us,E,1,1,train,Threeking RC Cars Stunt car Remote Control Car...,Rc Car Stunt Car Remote Control Car Great Gift...,Amazing STUNT CAR:In addition to the basic ski...,Threeking,Green


In [ ]:
def drop_null_values(df, columns, drop_na=True, seed=2025):
    """
    Drop null values from specified columns of a DataFrame.
    
    Args:
        df (pd.DataFrame): The DataFrame to process.
        columns (list): List of column names to check for null values.
        drop_na (bool): Whether to drop rows with any null values.
        seed (int): Random seed for reproducibility.
    """
    # Create sample dataset: ~500 rows with ~50 unique queries
    np.random.seed(seed) # set random seed for reproducibility (like to use the year)
    df_dropped = df.dropna(subset=['product_description', 'product_bullet_point', 'product_brand', 'product_color']).reset_index(drop=True)

    unique_queries = df_dropped['query'].unique()

    # Sample 50 queries randomly
    n_queries = min(50, len(unique_queries))
    sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)

    # Get all products for these queries
    df_sample = df_dropped[df_dropped['query'].isin(sample_queries)].copy()

    # If too many rows, sample proportionally per query
    if df_sample.shape[0] > 500:
        target_per_query = max(1, 500 // n_queries)
        df_sample = df_sample.groupby('query').apply(
            lambda x: x.sample(n=min(len(x), target_per_query), random_state=2025)
        ).reset_index(drop=True)
    else:
        df_sample = df_sample.reset_index(drop=True)

    print(f"Final sample: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

    # Look at the sample dataset head
    # df_sample[['query', 'product_title']].head()
    df_sample.head()

In [ ]:
# Save the sample dataset
df_sample.to_parquet("semantic_search_dataset.parquet")
df_sample.to_csv("semantic_search_dataset.csv", index=False)

print(f"Dataset saved: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")
print("Files: semantic_search_dataset.parquet, semantic_search_dataset.csv")

In [ ]:
# Quick exploration of the final dataset
print("Dataset columns:")
print(df_sample.columns.tolist())
print(f"\nExample queries:")
for query in df_sample['query'].unique()[:3]:
    print(f"- {query}")

In [25]:
def create_sample_dataset(df, target_queries=50, target_rows=500, seed=2025):
    """
    Create a sample dataset following the assessment instructions.
    
    Args:
        df (pd.DataFrame): The filtered dataframe to sample from
        target_queries (int): Target number of unique queries (default: 50)
        target_rows (int): Target number of rows (default: 500) 
        seed (int): Random seed for reproducibility (default: 2025)
    
    Returns:
        pd.DataFrame: Sample dataset
    """
    # Set random seed for reproducibility
    np.random.seed(seed)
    
    # Step 1: Get all unique queries from the dataset
    unique_queries = df['query'].unique()
    print(f"Total unique queries available: {len(unique_queries)}")
    
    # Step 2: Randomly sample the target number of queries
    n_queries = min(target_queries, len(unique_queries))
    sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)
    print(f"Sampled {n_queries} unique queries")
    
    # Step 3: Filter dataset to contain only the sampled queries
    df_query_filtered = df[df['query'].isin(sample_queries)].copy()
    print(f"Dataset after query filtering: {df_query_filtered.shape[0]} rows")
    
    # Step 4: If we have more than target_rows, sample down to target_rows
    if df_query_filtered.shape[0] > target_rows:
        print(f"Sampling down from {df_query_filtered.shape[0]} to {target_rows} rows...")
        
        # Calculate how many rows per query to get close to target_rows
        rows_per_query = target_rows // n_queries
        remainder = target_rows % n_queries
        
        # Sample proportionally per query
        sampled_dfs = []
        queries_list = list(sample_queries)
        
        for i, query in enumerate(queries_list):
            query_df = df_query_filtered[df_query_filtered['query'] == query]
            
            # Give some queries one extra row to handle remainder
            n_rows_for_query = rows_per_query + (1 if i < remainder else 0)
            n_rows_for_query = min(n_rows_for_query, len(query_df))
            
            if n_rows_for_query > 0:
                sampled_query_df = query_df.sample(n=n_rows_for_query, random_state=seed+i)
                sampled_dfs.append(sampled_query_df)
        
        df_sample = pd.concat(sampled_dfs, ignore_index=True)
    else:
        df_sample = df_query_filtered.reset_index(drop=True)
    
    print(f"Final sample dataset: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} unique queries")
    
    # Show missing value counts for product columns
    product_cols = ['product_description', 'product_bullet_point', 'product_brand', 'product_color']
    print(f"\nMissing values in product columns:")
    for col in product_cols:
        if col in df_sample.columns:
            missing_count = df_sample[col].isna().sum()
            missing_pct = (missing_count / len(df_sample)) * 100
            print(f"  {col}: {missing_count} ({missing_pct:.1f}%)")
    
    return df_sample

# Create the sample dataset using the assessment instructions
print("Creating sample dataset from df_filtered...")
df_assessment_sample = create_sample_dataset(df_filtered, target_queries=50, target_rows=500, seed=2025)

# Display first few rows to verify
print(f"\nFirst 5 rows of the sample dataset:")
df_assessment_sample.head()


Creating sample dataset from df_filtered...
Total unique queries available: 29843
Sampled 50 unique queries
Dataset after query filtering: 407 rows
Final sample dataset: 407 rows, 50 unique queries

Missing values in product columns:
  product_description: 230 (56.5%)
  product_bullet_point: 84 (20.6%)
  product_brand: 39 (9.6%)
  product_color: 165 (40.5%)

First 5 rows of the sample dataset:


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,1673,#pinkandproud not sorry glitter lipstick,53,B07PGQDJ73,us,E,1,1,train,"Maybelline SuperStay Ink Crayon Lipstick, Matt...",None,Long Lasting Lip Color: Our SuperStay Ink Cray...,Maybelline New York,65 SETTLE FOR MORE
1,1676,#pinkandproud not sorry glitter lipstick,53,B07R7V7XLN,us,E,1,1,train,Coosa Glitter Liquid Lipsticks Set 6 color Dia...,<b>Coosa Glitter Shimmer Diamond shimmer glitt...,High Shimmering Pigments--Diamond shimmer glit...,Coosa,6 PCS
2,1685,#pinkandproud not sorry glitter lipstick,53,B08F7CWDY2,us,E,1,1,train,"Matte Liquid Lipstick Set, Durable Nude Lip Gl...",None,Healthy Beauty : Our lip gloss kit is hypoalle...,LANGMANNI,24PCS
3,1688,#pinkandproud not sorry glitter lipstick,53,B08LYP272Y,us,E,1,1,train,evpct 4Pack Mini Glitter Cigarette Lipstick Se...,<b>Specification: <b><br> ●6 Different Types o...,❤Cool Girl Cigarette Box Design -- This lipsti...,evpct,A-Set04
4,2958,$3what can you buy without siri dollar toy,96,B07GWPVCZK,us,E,1,1,train,Threeking RC Cars Stunt car Remote Control Car...,Rc Car Stunt Car Remote Control Car Great Gift...,Amazing STUNT CAR:In addition to the basic ski...,Threeking,Green


In [27]:
def create_sample_dataset_flexible(df, target_queries=50, target_rows=500, seed=2025, 
                                   prioritize='rows', max_queries=100):
    """
    Create a sample dataset with flexible constraints to meet both row and query targets.
    
    Args:
        df (pd.DataFrame): The filtered dataframe to sample from
        target_queries (int): Target number of unique queries (default: 50)
        target_rows (int): Target number of rows (default: 500) 
        seed (int): Random seed for reproducibility (default: 2025)
        prioritize (str): 'rows' to prioritize hitting row target, 'queries' to prioritize query target
        max_queries (int): Maximum number of queries to sample when prioritizing rows
    
    Returns:
        pd.DataFrame: Sample dataset
    """
    np.random.seed(seed)
    
    # Get query-level statistics to help with sampling
    query_counts = df['query'].value_counts()
    unique_queries = query_counts.index.tolist()
    print(f"Total unique queries available: {len(unique_queries)}")
    print(f"Query product counts - Min: {query_counts.min()}, Max: {query_counts.max()}, Mean: {query_counts.mean():.1f}")
    
    if prioritize == 'rows':
        print(f"\\nPrioritizing hitting {target_rows} rows target...")
        
        # Strategy: Start with target_queries, increase if needed to hit row target
        current_queries = min(target_queries, len(unique_queries))
        
        for attempt_queries in range(current_queries, min(max_queries + 1, len(unique_queries) + 1)):
            # Sample queries
            sample_queries = np.random.choice(unique_queries, size=attempt_queries, replace=False)
            df_query_filtered = df[df['query'].isin(sample_queries)].copy()
            
            print(f"  Trying {attempt_queries} queries: {df_query_filtered.shape[0]} total rows")
            
            if df_query_filtered.shape[0] >= target_rows:
                print(f"  ✓ Found {attempt_queries} queries with {df_query_filtered.shape[0]} rows (>= {target_rows})")
                
                # Now sample down to exactly target_rows
                if df_query_filtered.shape[0] > target_rows:
                    df_sample = df_query_filtered.sample(n=target_rows, random_state=seed).reset_index(drop=True)
                else:
                    df_sample = df_query_filtered.reset_index(drop=True)
                
                break
        else:
            # If we couldn't hit target_rows even with max_queries, use what we have
            print(f"  ⚠ Could not reach {target_rows} rows even with {max_queries} queries")
            print(f"    Using all {df_query_filtered.shape[0]} rows from {attempt_queries} queries")
            df_sample = df_query_filtered.reset_index(drop=True)
    
    else:  # prioritize == 'queries'
        print(f"\\nPrioritizing hitting {target_queries} queries target...")
        
        # Original strategy: sample exactly target_queries, then work with available rows
        n_queries = min(target_queries, len(unique_queries))
        sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)
        df_query_filtered = df[df['query'].isin(sample_queries)].copy()
        
        print(f"Sampled {n_queries} queries with {df_query_filtered.shape[0]} total rows")
        
        if df_query_filtered.shape[0] > target_rows:
            # Sample down proportionally per query
            rows_per_query = target_rows // n_queries
            remainder = target_rows % n_queries
            
            sampled_dfs = []
            queries_list = list(sample_queries)
            
            for i, query in enumerate(queries_list):
                query_df = df_query_filtered[df_query_filtered['query'] == query]
                n_rows_for_query = rows_per_query + (1 if i < remainder else 0)
                n_rows_for_query = min(n_rows_for_query, len(query_df))
                
                if n_rows_for_query > 0:
                    sampled_query_df = query_df.sample(n=n_rows_for_query, random_state=seed+i)
                    sampled_dfs.append(sampled_query_df)
            
            df_sample = pd.concat(sampled_dfs, ignore_index=True)
        else:
            df_sample = df_query_filtered.reset_index(drop=True)
    
    print(f"\\nFinal sample dataset: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} unique queries")
    
    # Show missing value counts for product columns
    product_cols = ['product_description', 'product_bullet_point', 'product_brand', 'product_color']
    print(f"\\nMissing values in product columns:")
    for col in product_cols:
        if col in df_sample.columns:
            missing_count = df_sample[col].isna().sum()
            missing_pct = (missing_count / len(df_sample)) * 100
            print(f"  {col}: {missing_count} ({missing_pct:.1f}%)")
    
    return df_sample

# Try the flexible approach prioritizing rows to hit the 500 target
print("=== APPROACH 1: Prioritizing 500 rows (may use more than 50 queries) ===")
df_sample_rows = create_sample_dataset_flexible(
    df_filtered, 
    target_queries=50, 
    target_rows=500, 
    seed=2025, 
    prioritize='rows',
    max_queries=100
)

print(f"\\n" + "="*70)
print("=== APPROACH 2: Prioritizing exactly 50 queries (may get fewer than 500 rows) ===")
df_sample_queries = create_sample_dataset_flexible(
    df_filtered, 
    target_queries=50, 
    target_rows=500, 
    seed=2025, 
    prioritize='queries'
)

print(f"\\n" + "="*70)
print("SUMMARY:")
print(f"Approach 1 (prioritize rows): {df_sample_rows.shape[0]} rows, {df_sample_rows['query'].nunique()} queries")
print(f"Approach 2 (prioritize queries): {df_sample_queries.shape[0]} rows, {df_sample_queries['query'].nunique()} queries")


=== APPROACH 1: Prioritizing 500 rows (may use more than 50 queries) ===
Total unique queries available: 29843
Query product counts - Min: 1, Max: 81, Mean: 8.8
\nPrioritizing hitting 500 rows target...
  Trying 50 queries: 494 total rows
  Trying 51 queries: 528 total rows
  ✓ Found 51 queries with 528 rows (>= 500)
\nFinal sample dataset: 500 rows, 51 unique queries
\nMissing values in product columns:
  product_description: 222 (44.4%)
  product_bullet_point: 51 (10.2%)
  product_brand: 8 (1.6%)
  product_color: 143 (28.6%)
\n======================================================================
=== APPROACH 2: Prioritizing exactly 50 queries (may get fewer than 500 rows) ===
Total unique queries available: 29843
Query product counts - Min: 1, Max: 81, Mean: 8.8
\nPrioritizing hitting 50 queries target...
Sampled 50 queries with 494 total rows
\nFinal sample dataset: 494 rows, 50 unique queries
\nMissing values in product columns:
  product_description: 267 (54.0%)
  product_bullet_

In [ ]:
df_sample.info()

In [26]:
df_assessment_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407 entries, 0 to 406
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   example_id            407 non-null    int64 
 1   query                 407 non-null    object
 2   query_id              407 non-null    int64 
 3   product_id            407 non-null    object
 4   product_locale        407 non-null    object
 5   esci_label            407 non-null    object
 6   small_version         407 non-null    int64 
 7   large_version         407 non-null    int64 
 8   split                 407 non-null    object
 9   product_title         407 non-null    object
 10  product_description   177 non-null    object
 11  product_bullet_point  323 non-null    object
 12  product_brand         368 non-null    object
 13  product_color         242 non-null    object
dtypes: int64(4), object(10)
memory usage: 44.6+ KB
